In [1]:
# !pip install s3fs
# !pip install ../dist/aidial_rag_eval-0.1.0-py3-none-any.whl

In [2]:
import json
import pandas as pd
from aidial_rag_eval.evaluate import evaluate

/mnt/c/Users/Mikalai_Pechaneu/Work/dial-rag-eval/nli-branch-new/dial-rag-eval/src/aidial_rag_eval/generation/utils/progress_bar.py:5: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [3]:
ground_truth = "./example_parquets/ground_truth.parquet"
answers = "./example_parquets/answer.parquet"
metrics = "./example_parquets/metrics.parquet"

In [4]:
pd.read_parquet(ground_truth).head()

,question,documents,facts,answer
0,Who was the first president of the United States?,[george_washington],[George Washington was an American political l...,The first president of the United States was G...
1,When was Apple Inc. established and who was th...,[apple.pdf],"[Apple Inc. was established on April 1, 1976.,...",The establishment date of Apple Inc. is April ...
2,What is the capital city of Spain?,[spain],[Capital city of Spain is Madrid.],The capital city of Spain is Madrid.


In [5]:
pd.read_parquet(answers).head()

,question,documents,context,answer
0,Who was the first president of the United States?,[george_washington],[George Washington was an American political l...,George Washington was the first president of t...
1,When was Apple Inc. established and who was th...,[apple.pdf],[Apple Inc. is an American multinational techn...,The CEO of Apple in 2020 was Tim Cook.
2,What is the capital city of Spain?,[spain],[Barcelona is a city in Spain. It is the capit...,The capital city of Spain is Barcelona.


In [6]:
metrics_dataset = evaluate(ground_truth, answers, metrics)
print(metrics_dataset.data_path)
print(metrics_dataset.metadata.metrics)

./example_parquets/metrics.parquet
{'recall': 0.5, 'precision': 0.3333333333333333, 'mrr': 0.4166666666666667, 'f1': 0.38888888888888884}


In [7]:
metrics_df = metrics_dataset.read_dataframe()
metrics_df.head()

,question,documents,facts,ground_truth_answer,context,answer,facts_ranks,context_relevance,context_highlight,recall,precision,mrr,f1
0,Who was the first president of the United States?,[george_washington],[George Washington was an American political l...,The first president of the United States was G...,[George Washington was an American political l...,George Washington was the first president of t...,[0],"[1, 0]","[{""match"": [{""text"": ""George Washington was an...",1.0,0.5,1.00,0.666667
1,When was Apple Inc. established and who was th...,[apple.pdf],"[Apple Inc. was established on April 1, 1976.,...",The establishment date of Apple Inc. is April ...,[Apple Inc. is an American multinational techn...,The CEO of Apple in 2020 was Tim Cook.,"[-1, 1]","[0, 1]","[{""match"": [{""text"": ""Apple Inc. is an America...",0.5,0.5,0.25,0.500000
2,What is the capital city of Spain?,[spain],[Capital city of Spain is Madrid.],The capital city of Spain is Madrid.,[Barcelona is a city in Spain. It is the capit...,The capital city of Spain is Barcelona.,[-1],"[0, 0]","[{""match"": [{""text"": ""Barcelona is a city in S...",0.0,0.0,0.00,0.000000


In [8]:
metrics_df[["facts", "context", "facts_ranks", "context_relevance"]].head()

,facts,context,facts_ranks,context_relevance
0,[George Washington was an American political l...,[George Washington was an American political l...,[0],"[1, 0]"
1,"[Apple Inc. was established on April 1, 1976.,...",[Apple Inc. is an American multinational techn...,"[-1, 1]","[0, 1]"
2,[Capital city of Spain is Madrid.],[Barcelona is a city in Spain. It is the capit...,[-1],"[0, 0]"


In [9]:
metrics_df[["facts_ranks", "context_relevance", "recall", "precision", "mrr", "f1"]].head()

,facts_ranks,context_relevance,recall,precision,mrr,f1
0,[0],"[1, 0]",1.0,0.5,1.00,0.666667
1,"[-1, 1]","[0, 1]",0.5,0.5,0.25,0.500000
2,[-1],"[0, 0]",0.0,0.0,0.00,0.000000


In [10]:
print(metrics_df["context"].iloc[1][1])

In 2020, the CEO of Apple was Tim Cook.


In [11]:
print(json.dumps(json.loads(metrics_df["context_highlight"].iloc[1][1]), indent=2))

{
  "match": [
    {
      "text": "In 2020, the CEO of Apple was Tim Cook.",
      "facts": [
        1
      ]
    }
  ]
}
